# Manual Annotation Analysis

This notebook:
1. Converts each sheet in the Excel file to individual CSV files
2. Analyzes repeated tweets in each CSV

In [1]:
import pandas as pd
import os
from pathlib import Path

## 1. Convert Excel Sheets to CSV Files

In [2]:
# Define paths
xlsx_path = 'FInal Annotations sheet.xlsx'
csv_output_dir = 'csv_sheets'

# Create output directory if it doesn't exist
os.makedirs(csv_output_dir, exist_ok=True)

# Read Excel file and get all sheet names
xl = pd.ExcelFile(xlsx_path)
print(f"Found {len(xl.sheet_names)} sheets:")
for sheet in xl.sheet_names:
    print(f"  - {sheet}")

Found 15 sheets:
  - farmers protests
  - hindutva
  - kashmiri pandits
  - congress
  - hindu
  - kashmir
  - rahulgandhi
  - china
  - farm laws
  - caa
  - new parliament
  - shaheen bagh
  - ram mandir
  - modi
  - muslim


In [3]:
# Convert each sheet to CSV
csv_files = []

for sheet_name in xl.sheet_names:
    # Read the sheet
    df = pd.read_excel(xlsx_path, sheet_name=sheet_name)
    
    # Create a safe filename (replace spaces and special chars)
    safe_name = sheet_name.replace(' ', '_').replace('/', '_').replace('\\', '_')
    csv_filename = f"{safe_name}.csv"
    csv_path = os.path.join(csv_output_dir, csv_filename)
    
    # Save to CSV
    df.to_csv(csv_path, index=False)
    csv_files.append(csv_path)
    
    print(f"✓ Saved '{sheet_name}' -> {csv_path} ({len(df)} rows)")

print(f"\n✅ Successfully converted {len(csv_files)} sheets to CSV files in '{csv_output_dir}/'")

✓ Saved 'farmers protests' -> csv_sheets/farmers_protests.csv (120 rows)
✓ Saved 'hindutva' -> csv_sheets/hindutva.csv (120 rows)
✓ Saved 'kashmiri pandits' -> csv_sheets/kashmiri_pandits.csv (120 rows)
✓ Saved 'congress' -> csv_sheets/congress.csv (120 rows)
✓ Saved 'hindu' -> csv_sheets/hindu.csv (120 rows)
✓ Saved 'kashmir' -> csv_sheets/kashmir.csv (120 rows)
✓ Saved 'rahulgandhi' -> csv_sheets/rahulgandhi.csv (120 rows)
✓ Saved 'china' -> csv_sheets/china.csv (120 rows)
✓ Saved 'farm laws' -> csv_sheets/farm_laws.csv (150 rows)
✓ Saved 'caa' -> csv_sheets/caa.csv (150 rows)
✓ Saved 'new parliament' -> csv_sheets/new_parliament.csv (150 rows)
✓ Saved 'shaheen bagh' -> csv_sheets/shaheen_bagh.csv (150 rows)
✓ Saved 'ram mandir' -> csv_sheets/ram_mandir.csv (150 rows)
✓ Saved 'modi' -> csv_sheets/modi.csv (150 rows)
✓ Saved 'muslim' -> csv_sheets/muslim.csv (150 rows)

✅ Successfully converted 15 sheets to CSV files in 'csv_sheets/'


## 2. Analyze Repeated Tweets in Each CSV

In [4]:
# First, let's look at the columns in one of the CSVs to identify the tweet column
sample_df = pd.read_csv(csv_files[0])
print("Columns in the data:")
print(sample_df.columns.tolist())
print("\nFirst few rows:")
sample_df.head()

Columns in the data:
['source_row', 'tweet', 'tweet_label', 'subjects', 'subjects_scored', 'keyword', 'label_norm', 'matched keyword', 'stance ', 'stance reason']

First few rows:


,source_row,tweet,tweet_label,subjects,subjects_scored,keyword,label_norm,matched keyword,stance,stance reason
0,717821,Those asking for political rallies to stop &am...,Pro Ruling,"['protests', 'farmers', 'spreaders']","[{'text': 'protests', 'score': 0.3905}, {'text...",farmers protests,Pro Ruling,farmers protests,Against,farmers protest is a super spreader of covid.
1,1487834,"Breaking: India reacts to comments by Rihanna,...",Pro Ruling,"['india', 'protests', 'hashtags']","[{'text': 'india', 'score': 0.4438}, {'text': ...",farmers protests,Pro Ruling,farmers protests,Against,condems foreign celebrities for sensationalisi...
2,571770,Protesters heckle Republic Bharat and Zee News...,Pro Ruling,"['protesters', 'journalists', 'farmers']","[{'text': 'protesters', 'score': 0.5599}, {'te...",farmers protests,Pro Ruling,farmers protests,neutral,no stance taken.
3,1012783,Khalistani Sponsored Farmers’ Protests Links W...,Pro Ruling,"['protests', 'farmers', 'shell']","[{'text': 'protests', 'score': 0.4749}, {'text...",farmers protests,Pro Ruling,farmers protests,Against,accuses farmers protest of being sponsored by ...
4,543384,The above four incidents have happened in the ...,Pro Ruling,"['pakistan', 'protests', 'farmers']","[{'text': 'pakistan', 'score': 0.5321}, {'text...",farmers protests,Pro Ruling,farmers protests,Against,says farmers protest is being used by Pakistan...


In [5]:
# Function to analyze repeated tweets in a CSV
def analyze_repeated_tweets(csv_path, tweet_column=None):
    df = pd.read_csv(csv_path)
    
    if tweet_column is None:
        possible_names = ['tweet', 'text', 'content', 'Tweet', 'Text', 'Content', 
                          'tweet_text', 'full_text', 'body', 'message']
        for col in possible_names:
            if col in df.columns:
                tweet_column = col
                break
        if tweet_column is None:
            for col in df.columns:
                if df[col].dtype == 'object':
                    tweet_column = col
                    break
    
    if tweet_column is None or tweet_column not in df.columns:
        return {'error': f'Could not find tweet column in {csv_path}'}
    
    total_tweets = len(df)
    unique_tweets = df[tweet_column].nunique()
    repeated_tweets = total_tweets - unique_tweets
    
    value_counts = df[tweet_column].value_counts()
    duplicates = value_counts[value_counts > 1]
    
    return {
        'file': os.path.basename(csv_path),
        'tweet_column': tweet_column,
        'total_rows': total_tweets,
        'unique_tweets': unique_tweets,
        'repeated_count': repeated_tweets,
        'duplicate_percentage': (repeated_tweets / total_tweets * 100) if total_tweets > 0 else 0,
        'num_duplicate_groups': len(duplicates),
        'top_duplicates': duplicates.head(10).to_dict() if len(duplicates) > 0 else {}
    }

In [6]:
# Analyze all CSV files
results = []

for csv_path in csv_files:
    result = analyze_repeated_tweets(csv_path)
    results.append(result)

# Create summary DataFrame
summary_df = pd.DataFrame(results)
print("=" * 80)
print("REPEATED TWEETS ANALYSIS SUMMARY")
print("=" * 80)
summary_df[['file', 'total_rows', 'unique_tweets', 'repeated_count', 'duplicate_percentage']]

REPEATED TWEETS ANALYSIS SUMMARY


,file,total_rows,unique_tweets,repeated_count,duplicate_percentage
0,farmers_protests.csv,120,102,18,15.000000
1,hindutva.csv,120,116,4,3.333333
2,kashmiri_pandits.csv,120,94,26,21.666667
3,congress.csv,120,118,2,1.666667
4,hindu.csv,120,120,0,0.000000
5,kashmir.csv,120,116,4,3.333333
6,rahulgandhi.csv,120,113,7,5.833333
7,china.csv,120,119,1,0.833333
8,farm_laws.csv,150,150,0,0.000000
9,caa.csv,150,150,0,0.000000


In [7]:
# Detailed output for each file
print("\n" + "=" * 80)
print("DETAILED ANALYSIS BY FILE")
print("=" * 80)

for result in results:
    if 'error' in result:
        print(f"\n❌ {result.get('file', 'Unknown')}: {result['error']}")
        continue
        
    print(f"\n📄 File: {result['file']}")
    print(f"   Tweet column: '{result['tweet_column']}'")
    print(f"   Total rows: {result['total_rows']}")
    print(f"   Unique tweets: {result['unique_tweets']}")
    print(f"   Repeated tweets: {result['repeated_count']} ({result['duplicate_percentage']:.2f}%)")
    
    if result['top_duplicates']:
        print(f"   Duplicate groups: {result['num_duplicate_groups']}")
        print("   Top duplicates (tweet preview -> count):")
        for tweet, count in list(result['top_duplicates'].items())[:5]:
            preview = str(tweet)[:50] + "..." if len(str(tweet)) > 50 else str(tweet)
            print(f"      - '{preview}' appears {count} times")


DETAILED ANALYSIS BY FILE

📄 File: farmers_protests.csv
   Tweet column: 'tweet'
   Total rows: 120
   Unique tweets: 102
   Repeated tweets: 18 (15.00%)
   Duplicate groups: 12
   Top duplicates (tweet preview -> count):
      - 'Farmers protests. 
 
 Free liquor distribution htt...' appears 5 times
      - 'Bhartiya Kisan Union is leading organiser of farme...' appears 4 times
      - 'The ugly truth of "Farmers" protests https://t.co/...' appears 3 times
      - 'Those asking for political rallies to stop &amp; k...' appears 2 times
      - 'Deep Sidhu arrested!
 
 So if he was a BJP agent w...' appears 2 times

📄 File: hindutva.csv
   Tweet column: 'tweet'
   Total rows: 120
   Unique tweets: 116
   Repeated tweets: 4 (3.33%)
   Duplicate groups: 4
   Top duplicates (tweet preview -> count):
      - 'Dear @narendramodi ji,
 
 Since last 7 years, 
 I'...' appears 2 times
      - 'When Sangh says Hindusthan is Hindu Rashtra it doe...' appears 2 times
      - 'Delhi Riot was not a ri

In [8]:
# Save the summary to CSV
summary_output_path = os.path.join(csv_output_dir, '_analysis_summary.csv')
summary_df.to_csv(summary_output_path, index=False)
print(f"\n✅ Summary saved to: {summary_output_path}")


✅ Summary saved to: csv_sheets/_analysis_summary.csv


## 3. Optional: Get All Repeated Tweets with Details

In [9]:
def get_all_duplicates_detailed(csv_path, tweet_column=None):
    df = pd.read_csv(csv_path)
    
    if tweet_column is None:
        possible_names = ['tweet', 'text', 'content', 'Tweet', 'Text', 'Content', 
                          'tweet_text', 'full_text', 'body', 'message']
        for col in possible_names:
            if col in df.columns:
                tweet_column = col
                break
        if tweet_column is None:
            for col in df.columns:
                if df[col].dtype == 'object':
                    tweet_column = col
                    break
    
    duplicated_mask = df.duplicated(subset=[tweet_column], keep=False)
    duplicates_df = df[duplicated_mask].sort_values(tweet_column)
    
    return duplicates_df, tweet_column

# Example: View duplicates from the first CSV
if csv_files:
    duplicates_df, col = get_all_duplicates_detailed(csv_files[0])
    print(f"Duplicates from {os.path.basename(csv_files[0])} (using column '{col}'):")
    print(f"Total duplicate rows: {len(duplicates_df)}")
    if len(duplicates_df) > 0:
        duplicates_df.head(10)

Duplicates from farmers_protests.csv (using column 'tweet'):
Total duplicate rows: 30


## 4. Remove Repeated Tweets and Save Clean CSVs

This section removes all duplicate tweets (keeping only one of each) and saves the cleaned CSVs to a new folder.

In [10]:
# Output folder for deduplicated CSVs
dedup_output_dir = 'no repetitions before finetuning'
os.makedirs(dedup_output_dir, exist_ok=True)

print("=" * 80)
print("REMOVING REPEATED TWEETS")
print("=" * 80)

dedup_results = []

for csv_path in csv_files:
    csv_file = os.path.basename(csv_path)
    
    # Read the CSV
    df = pd.read_csv(csv_path)
    original_count = len(df)
    
    # Remove duplicates based on 'tweet' column, keep first occurrence
    df_deduped = df.drop_duplicates(subset=['tweet'], keep='first')
    deduped_count = len(df_deduped)
    removed_count = original_count - deduped_count
    
    # Save to new folder
    output_path = os.path.join(dedup_output_dir, csv_file)
    df_deduped.to_csv(output_path, index=False)
    
    dedup_results.append({
        'file': csv_file,
        'original_count': original_count,
        'deduped_count': deduped_count,
        'removed_count': removed_count
    })
    
    if removed_count > 0:
        print(f"📄 {csv_file}: {original_count} → {deduped_count} (removed {removed_count} duplicates)")
    else:
        print(f"✓ {csv_file}: {original_count} rows (no duplicates)")

# Summary
dedup_summary_df = pd.DataFrame(dedup_results)
total_removed = dedup_summary_df['removed_count'].sum()

print("\n" + "=" * 80)
print(f"SUMMARY: Removed {total_removed} total duplicate tweets")
print(f"✅ Cleaned CSVs saved to: {dedup_output_dir}/")
print("=" * 80)

REMOVING REPEATED TWEETS
📄 farmers_protests.csv: 120 → 102 (removed 18 duplicates)
📄 hindutva.csv: 120 → 116 (removed 4 duplicates)
📄 kashmiri_pandits.csv: 120 → 94 (removed 26 duplicates)
📄 congress.csv: 120 → 118 (removed 2 duplicates)
✓ hindu.csv: 120 rows (no duplicates)
📄 kashmir.csv: 120 → 116 (removed 4 duplicates)
📄 rahulgandhi.csv: 120 → 113 (removed 7 duplicates)
📄 china.csv: 120 → 119 (removed 1 duplicates)
✓ farm_laws.csv: 150 rows (no duplicates)
✓ caa.csv: 150 rows (no duplicates)
✓ new_parliament.csv: 150 rows (no duplicates)
✓ shaheen_bagh.csv: 150 rows (no duplicates)
✓ ram_mandir.csv: 150 rows (no duplicates)
📄 modi.csv: 150 → 146 (removed 4 duplicates)
✓ muslim.csv: 150 rows (no duplicates)

SUMMARY: Removed 66 total duplicate tweets
✅ Cleaned CSVs saved to: no repetitions before finetuning/


In [11]:
# Show summary table
print("\nDETAILED SUMMARY:")
dedup_summary_df


DETAILED SUMMARY:


,file,original_count,deduped_count,removed_count
0,farmers_protests.csv,120,102,18
1,hindutva.csv,120,116,4
2,kashmiri_pandits.csv,120,94,26
3,congress.csv,120,118,2
4,hindu.csv,120,120,0
5,kashmir.csv,120,116,4
6,rahulgandhi.csv,120,113,7
7,china.csv,120,119,1
8,farm_laws.csv,150,150,0
9,caa.csv,150,150,0
